# Iris Visualization

## Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Spark libs
from pyspark.sql.session import SparkSession

# helper functions
from helpers.helper_functions import translate_to_file_string

Select the Imput File

In [ ]:
inputFile = translate_to_file_string("../data/iris.data")

SparkSession creation

In [ ]:
spark = (SparkSession
       .builder
       .appName("IrisVisualization")
       .getOrCreate())

Create a DataFrame using an ifered schema 

In [ ]:
# load data file.
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "false") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(inputFile) \
       .withColumnRenamed("_c0","sepal length")\
       .withColumnRenamed("_c1","sepal width") \
       .withColumnRenamed("_c2","petal length")\
       .withColumnRenamed("_c3","petal width") \
       .withColumnRenamed("_c4","label")
print(df.printSchema())

## Plot the data

In [ ]:
printDf(df.summary())

In [ ]:
### Box Plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df.toPandas().boxplot(sym='r+', figsize=(20,15))
save_fig("attribute_box_plots")
plt.show()

### Histograms

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df.toPandas().hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots")
plt.show()

### Density Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df.toPandas().plot.kde(figsize=(20,15))
save_fig("attribute_density_plots")
plt.show()

### Scatter Plot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
ax = df.filter(df.label == "Iris-setosa" ).toPandas().plot.scatter(x='sepal length', y='petal width', color='DarkBlue', label='Iris-setosa')
df.filter(df.label == "Iris-versicolor").toPandas().plot.scatter(x='sepal length', y='petal width', color='DarkGreen', label='Iris-versicolour', ax=ax)
df.filter(df.label == "Iris-virginica").toPandas().plot.scatter(x='sepal length', y='petal width', color='DarkRed', label='Iris-virginica', ax=ax)
save_fig("scatter_plots")
plt.show()

### Scatter matrix

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
scatter_matrix(df.toPandas(), figsize=(30, 30), diagonal='kde')
save_fig("scatter_matrix")
plt.show()

### Bubble Charts

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
dfPandas= df.toPandas()
dfPandas.plot.scatter(x='sepal length', y='petal width', s=dfPandas['sepal width'] * 10, color='DarkBlue')
save_fig("bubble_plots")
plt.show()

### Parallel coordinates

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
parallel_coordinates(df.toPandas(), 'label')
save_fig("paralle_coordinates_plots")
plt.show()

In [ ]:
spark.stop()